# Price ML Predicion (5 Days out)

## Price Prediction's Caveats

   - Make predictions into line segments. Where the first point is aligned to the last close price, and the last point is the predicted close price. The last point is the delta between the prior predicted close (moved up to the last close price) and the predicted close price.
      1. Generate deltas between each predicted close price.
      2. Generate line segments from close price to close price + delta.
      3. Plot the line segments.
   
   - Another take that makes more sense...
     is to take the deltas generated between the current prediction and the prior prediction and apply that differential to the prior close.
     - Thus we are plotting the predictions from the prior close.
<p>

## Enhancements (to add)

We should be able to add prediction targets for high and low for the day by 
   adding the target high-difference and low-difference to the data set,
   similar to the way we added the target close-difference.
   
We can also apparently pull minute bars from Yahoo. And with that, we can
   perform more granular predictions, to the hour, 15min, 5min, and
   1min period.
   
## Version Notes:
   - The new way of adjusting predictions to the actual price movement works.
   - We apply the predictions delta to the new close once we know it. This make a lot more sense.
   - It also make a lot of sense that the predictions are really predictions of price movement, thus we should not expect the
     predictions to exactly match to the actual price movement.
   - Additionally, the adjustment is simple and very straight forward.
            

# Choose an existing model -or- Create a New Model


In [1]:
# Create a list of available .keras models from model_dir...
# Restart the kernel
%reset -f
%gui asyncio

import ipywidgets as widgets
from ipywidgets import Output, Checkbox, TwoByTwoLayout
import re
import datetime
import os
import asyncio
import yfinance as yf

# os.environ['TF_ENABLE_ONEDNN_OPTS'] = '0'

global Ticker, DataStart, DateEnd, ticker, dateStart, dateEnd, CreateNewModel, model_dir, model_files, Selected_Model

Ticker    = 'AMZN'

DateStart = '2015-06-03'
DateEnd   = '2024-07-30'
CreateNewModel = "Create New Model"

# Get list of file from the model directory...
model_dir = 'models/'
model_files = os.listdir(model_dir)
Selected_Model = None

def wait_for_change(widget, value):
    future = asyncio.Future()
    def getvalue(change):
        # make the new value available
        future.set_result(change.new)
        widget.unobserve(getvalue, value)
    widget.observe(getvalue, value)
    return future

PredsDaily  = 'Daily Predictions'
PredsWeekly = 'Weekly Predictions'

def model_choices():
    global Ticker, DataStart, DateEnd, ticker, dateStart, dateEnd, CreateNewModel, model_dir, model_files, Selected_Model
    
    # Filter the list to only include .keras files...
    models_lst = [file for file in model_files if re.search(r'\.keras$', file)]
    models_lst.sort()
    model_files = [CreateNewModel] + models_lst
    # Display the list of files...
    Selected_Model = widgets.Select ( options=model_files,
                                      value=model_files[0],
                                      description='Model File: ',
                                      disabled=False )
    ticker     = widgets.Text(value=Ticker, description='TICKER: ')
    DateStart = "2015-01-01"
    dateStart = widgets.Text(value=DateStart, description='Start Date: ')
    chk_save_model = widgets.Checkbox(value=True, description='Save Model')
    dd_period = widgets.Dropdown(options=[PredsDaily, PredsWeekly],
                                 value=PredsDaily, description='Periods')
    
    # Set the DateEnd to today's date with the format 'YYYY-MM-DD, determined by the system date.
    if Selected_Model.value == CreateNewModel:
        DateEnd = datetime.datetime.now().strftime('%Y-%m-%d')
    dateEnd   = widgets.Text(value=DateEnd, description='End Date: ')
    out = Output()
    
    async def f0():
        for i in range(10):
            out.append_stdout('did work ' + str(i) + '\n')
            x = await wait_for_change(Selected_Model, 'value')
            if Selected_Model.value != CreateNewModel:
                ticker.value = Selected_Model.value.split('_')[0]
                if '~' in ticker.value:
                    ticker.value = ticker.value.replace('~', '=')
            out.append_stdout('async function continued with value ' + str(x) + '\n')
        asyncio.ensure_future(f0())
    
    async def f1():
        for i in range(10):
            out.append_stdout('did work ' + str(i) + '\n')
            x = await wait_for_change(ticker, 'value')
            tkr = ticker.value
            tkr = re.sub(r'Invalid: ', '', tkr)
            if Selected_Model.value == CreateNewModel:
                # Check yahoo if symbol is valid...
                try:
                    ticker_ = yf.Ticker(tkr).history(period='5d',interval='1d')
                except:
                    ticker_ = []
                if len(ticker_) == 0:
                    ticker.value = 'Invalid: ' + tkr
                else:
                    ticker.value = ticker.upper()
            out.append_stdout('async function continued with value ' + str(x) + '\n')
        asyncio.ensure_future(f1())
    
    asyncio.ensure_future(f0())
    asyncio.ensure_future(f1())
    
    from IPython.display import display, Pretty, Markdown, Latex
    inst0 = """
        +-------------------------------+
        | Choose or Create a New Model  |
        +-------------------------------+
        
    Make you Model Selection and Updates the Start and End Dates.
    Or, Create a New Model by entering by leave the Model File: as
    "Create New Model", and entering the Ticker, Start Date and End Date.
    """
    display(Pretty(inst0))
    
    # print(inst0, inst1)
    display(Selected_Model)
    print()
    display(ticker, dateStart, dateEnd, chk_save_model, dd_period)
    print()
    # display(out)

    # if selected_model.value == CreateNewModel:
    # - Create a new model using the ticker.value, dateStart.value, DateEnd.vale values.
    return(Selected_Model, ticker, dateStart, dateEnd, chk_save_model, dd_period, asyncio)

Selected_Model, ticker, dateStart, dateEnd, chk_save_model, dd_period, asyncio = model_choices()
        
        
        


        +-------------------------------+
        | Choose or Create a New Model  |
        +-------------------------------+
        
    Make you Model Selection and Updates the Start and End Dates.
    Or, Create a New Model by entering by leave the Model File: as
    "Create New Model", and entering the Ticker, Start Date and End Date.
    

Select(description='Model File: ', options=('Create New Model', 'AAPL_D_2020-08-28_2024-08-26.keras', 'ABT_D_2…

Text(value='AMZN', description='TICKER: ')

Text(value='2015-01-01', description='Start Date: ')

Text(value='2024-08-28', description='End Date: ')

Checkbox(value=True, description='Save Model')

Dropdown(description='Periods', options=('Daily Predictions', 'Weekly Predictions'), value='Daily Predictions'…

In [2]:
def print_model_choices(Selected_Model, ticker, dateStart, dateEnd, chk_save_model, dd_period):
    print( "Selected_Model:", Selected_Model.value)
    print("Ticker: ", ticker.value)
    print("DateStart: ", dateStart.value)
    print("DateEnd: ", dateEnd.value)
    print("Save Model: ", chk_save_model.value)
    print("Period: ", dd_period.value)

print_model_choices(Selected_Model, ticker, dateStart, dateEnd, chk_save_model, dd_period)

Selected_Model: EGIO_D_2020-08-31_2024-08-27.keras
Ticker:  EGIO
DateStart:  2015-01-01
DateEnd:  2024-08-28
Save Model:  True
Period:  Daily Predictions


#  Load the latest data for the selected Ticker...

In [3]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn
import pandas as pd
import yfinance as yf

def load_data():
    global Ticker, DataStart, DateEnd, ticker, dateStart, dateEnd, CreateNewModel, model_dir, model_files, model_path, Selected_Model
    global data_orig, data, feature_cnt, feature_cnt_dl

    model_dir = 'models/'

    if Selected_Model.value == CreateNewModel:
        modelDateStart = DateStart
        modelDateEnd = DateEnd
        if dd_period.value == PredsDaily:
            model_file = ticker.value + "_D_" + dateStart.value + "_" + dateEnd.value + ".keras"
            model_path = model_dir + model_file
        elif dd_period.value == PredsWeekly:
            model_file = ticker.value + "_W_" + dateStart.value + "_" + dateEnd.value + ".keras"
            model_path = model_dir + model_file
            
    else:
        model_path = model_dir + Selected_Model.value

    # If we are using an existing model, we still have to download the data as we need the
    # actual price data to adjust the prediction.

    # If the predicted value varies/swings-about too much from the price,
    # the days/period training period many be changed to exclude the prior period
    # That does not have present-time characteristics with regard to price movement.
    data_orig = yf.download(tickers = ticker.value, start = dateStart.value, end = dateEnd.value)
    data=data_orig.copy(deep=True)
    offset = pd.Timedelta(days=-30)
    # Resample to 'W'eekly or 'ME'(Month End)
    # logic = {'Open'  : 'first',
    #          'High'  : 'max',
    #          'Low'   : 'min',
    #          'Close' : 'last',
    #          'Adj Close': 'last',
    #          'Volume': 'sum'}
    # data = data.resample('W', offset=offset).apply(logic)
    print("Pulling Data for:", ticker.value, "from", dateStart.value, "to", dateEnd.value)
    print("data.len():", len(data), "data.shape:", data.shape)

    feature_cnt_dl = data.shape[1]
    feature_cnt = feature_cnt_dl + 1  # Actual count vs index count.
    return data_orig, data, feature_cnt

def aggregate_data(data):
    data = data.resample('W').agg({'Open': 'first', 'High': 'max', 'Low': 'min', 'Close': 'last', 'Adj Close': 'last', 'Volume': 'sum'})
    return data

data_orig, data, feature_cnt = load_data()

# Keep a copy of the original data before possible aggregation.
data_original = data_orig

if dd_period.value == PredsWeekly or '_w_' in model_path:
    data = aggregate_data(data)
    data_orig = data.copy(deep=True)

print("data.shape:", data.shape)
print("data:", data.head(5), data.tail(5))


[*********************100%%**********************]  1 of 1 completed

Pulling Data for: EGIO from 2015-01-01 to 2024-08-28
data.len(): 2429 data.shape: (2429, 6)
data.shape: (2429, 6)
data:                   Open        High         Low       Close   Adj Close  Volume
Date                                                                          
2015-01-02  112.000000  112.800003  110.400002  110.800003  110.800003    2610
2015-01-05  110.800003  114.000000  110.800003  112.800003  112.800003    5135
2015-01-06  112.800003  112.800003  106.800003  108.400002  108.400002    5598
2015-01-07  108.800003  110.000000  105.599998  108.800003  108.800003    3325
2015-01-08  110.000000  112.000000  108.000000  111.199997  111.199997    3318              Open   High    Low  Close  Adj Close  Volume
Date                                                     
2024-08-21   8.50   8.60   8.30   8.35       8.35   38000
2024-08-22   8.32  10.47   8.32   9.89       9.89  173100
2024-08-23   9.80  11.00   9.80  10.76      10.76   87200
2024-08-26  10.94  11.17  10.20  10.7

##  - Add indicators to the data


In [4]:
# Add indicators to the data...

import pandas_ta as ta

def add_indicators(data, feature_cnt_dl):

    feature_cnt = feature_cnt_dl

    data = data_orig.copy(deep=True)

    # print("= Before Adding Indicators ========================================================")
    # print(data.tail(10))

    data['RSI']=ta.rsi(data.Close, length=3); feature_cnt += 1
    # data['EMAF']=ta.ema(data.Close, length=3); feature_cnt += 1
    # data['EMAM']=ta.ema(data.Close, length=6); feature_cnt += 1
    data['EMAS']=ta.ema(data.Close, length=9); feature_cnt += 1
    data['DPO3']=ta.dpo(data.Close, length=3, centered=True); feature_cnt += 1
    data['DPO6']=ta.dpo(data.Close, length=6, centered=True); feature_cnt += 1
    data['DPO9']=ta.dpo(data.Close, length=9, centered=True); feature_cnt += 1

    # print("= After Adding DPO2 ========================================================")
    # print(data.tail(10))
    #
    # On Balance Volume
    if data['Volume'].iloc[-1] > 0:
        data = data.join(ta.aobv(data.Close, data.Volume, fast=True, min_lookback=3, max_lookback=9))
        feature_cnt += 7 # ta.aobv adds 7 columns

    # print("= After Adding APBV ========================================================")
    # print(data.tail(10))

    # Target is the difference between the adjusted close and the open price.
    data['Target'] = data['Adj Close']-data.Open
    feature_cnt += 1
    data['TargetH'] = data.High - data.Open
    feature_cnt += 1
    data['TargetL'] = data.Low - data.Open
    feature_cnt += 1
    # Shift the target up by one day.Target is the difference between the adjusted close and the open price.
    # That is, the target is the difference between the adjusted close and the open price.
    # Our model will predict the target close for the next day. So we shift the target up by one day.
    data['Target'] = data['Target'].shift(-1)
    data['TargetH'] = data['TargetH'].shift(-1)
    data['TargetL'] = data['TargetL'].shift(-1)

    # 1 if the price goes up, 0 if the price goes down.
    # Not a feature: Needed to test prediction accuracy.
    data['TargetClass'] = [1 if data['Target'].iloc[i]>0 else 0 for i in range(len(data))]
    target_cnt = 1
    # The TargetNextClose is the adjusted close price for the next day.
    # This is the value we want to predict.
    # Not a feature: Needed to test prediction accuracy.
    data['TargetNextClose'] = data['Adj Close'].shift(-1)
    target_cnt += 1
    # TargetNextHigh and TargetNextLow are the high and low prices for the next day.
    data['TargetNextHigh'] = data['High'].shift(-1)
    target_cnt += 1
    # TargetNextLow are the low prices for the next day.
    data['TargetNextLow'] = data['Low'].shift(-1)
    target_cnt += 1

    # Before scaling the data, we need to use the last good value for rows that have NaN values.
    data.ffill(inplace=True)

    # print("= After Adding Targets___ and ForwardFill ========================================================")
    # print(data.tail(10))

    # Reset the index of the dataframe.
    data.reset_index(inplace = True)
    data_date = data['Date'].copy()
    data.drop(['Date'], axis=1, inplace=True);   feature_cnt -= 1
    data.drop(['Close'], axis=1, inplace=True);  feature_cnt -= 1
    # data.drop(['Volume'], axis=1, inplace=True); feature_cnt -= 1

    # Add one more row to the data file, this will be our next day's prediction.
    data = pd.concat([data,data[-1:]])
    # And, reindex the dataframe.
    data.reset_index(inplace=True)

    # print("feature_cnt_d1:", feature_cnt_dl, " feature_cnt:", feature_cnt)
    return data, feature_cnt, data_date

data_aug, feature_cnt, data_date = add_indicators(data, feature_cnt_dl)

data_set = data_aug.copy(deep=True)
print("Len data:", len(data_aug), "Len data_set", len(data_set))
# data_set.tail(10)

Len data: 2430 Len data_set 2430


## - Scale the data

  The data is scaled to a range of 0 to 1.
  This is done to ensure that the data is normalized, so that the model can be trained on it

In [5]:
# Scale the data for training...

import sklearn
from sklearn.preprocessing import MinMaxScaler


def scale_data(data_set):
    sc = MinMaxScaler(feature_range=(0,1))
    data_set_scaled = sc.fit_transform(data_set)
    return sc, data_set_scaled

sc, data_set_scaled = scale_data(data_set)

print("data_set.shape:", data_set.shape, "data_set_scaled.shape", data_set_scaled.shape)
# print(data_set_scaled)


data_set.shape: (2430, 25) data_set_scaled.shape (2430, 25)


In [6]:
# Check of nan an/or 0 dataums...
# After scaling the data, we need to check for NaN and 0 values,
# and forward fill the data with the last known value.
# In the case of a series of 0 values, begining at index 0,
# we have to back fill those values with the first non-non/zero value available.
# *** Currently, we are not handling this case. ***
def scaled_data_cleanup(data_set):
    nan_indices = np.argwhere(np.isnan(data_set_scaled))
    zer_indices = np.argwhere(data_set_scaled == 0)

    # print("nan_indices:", nan_indices)
    # print('zer_indices:', zer_indices)

    for i in range(len(nan_indices)):
        j = nan_indices[i][1] - 1
        if j < 0:
            j = nan_indices[i][1] + 1
        data_set_scaled[nan_indices[i][0], nan_indices[i][1]] = data_set_scaled[nan_indices[i][0], j]
    for i in range(len(zer_indices)):
        j = zer_indices[i][1] - 1
        if j < 0:
            j = zer_indices[i][1] + 1
        data_set_scaled[zer_indices[i][0], zer_indices[i][1]] = data_set_scaled[zer_indices[i][0], j]

    nan_indices = np.argwhere(np.isnan(data_set_scaled))
    zer_indices = np.argwhere(data_set_scaled == 0)

    return data_set_scaled, nan_indices, zer_indices

data_set_scaled, nan_indices, zer_indices = scaled_data_cleanup(data_set)
# print("nan_indices:", nan_indices)
# print('zer_indices:', zer_indices)
pass

## - Prepare the scaled data for the model

  - The data is prepared for the model by creating a 3D array of the data.
  - The data is split into training and test data.
  - The training data is used to train the model.
  - The test data is used to test the model.
  - The model is then used to predict the next day's price.

In [7]:
# LSTM needs a rolling period of data for each feature to make predictions.
# multiple feature from data provided to the model
global X, backcandles, y, yi, data_set
backcandles = 15 # Set the rolling window size.

def prep_model_inputs(data_scaled, backcandles):
    X = []
    # print(data_set_scaled[0].size)
    # data_set_scaled=data_set.values
    # print(data_set_scaled.shape[0])

    # Create a 3D array of the data. X[features][periods][candles]
    # Where candles is the number of candles that rolls by 1 period for each period.
    for j in range(feature_cnt): # data_set_scaled[0].size):# last 2 columns are target not X
        X.append([])
        for i in range(backcandles, data_set_scaled.shape[0]): # backcandles+2
            X[j].append(data_set_scaled[i-backcandles:i, j])

    # print("X.shape:", np.array(X).shape)
    X = np.array(X)
    # Move axis from 0 to position 2
    X=np.moveaxis(X, [0], [2])
    # print("X.shape:", X.shape)

    # Erase first elements of y because of backcandles to match X length
    # del(yi[0:backcandles])
    #X, yi = np.array(X), np.array(yi)
    # Choose -1 for last column, classification else -2...
    # Choose -4 for last 4 columns, classification (up/down), TargetNextClose, TargetNextHigh, and TargetNextLow ...
    X, yi = np.array(X), np.array(data_set_scaled[backcandles:, -4:])
    y = np.reshape(yi,(len(yi),4))
    #y=sc.fit_transform(yi)
    #X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))
    
    # print("data_set.shape:",data_set.shape,"X.shape:",X.shape)
    # print(X)
    # print("=========================================================")
    # print(y.shape)
    # print(y)

    return X, y, yi

X, y, yi = prep_model_inputs(data_set_scaled, backcandles)

pass

In [8]:
# split data into train test sets

global splitlimit
splitlimit = int(len(X)*0.8)

def split_data(X, y, splitlimit):
    # print("lenX:",len(X), "splitLimit:",splitlimit)
    X_train, X_test = X[:splitlimit], X[splitlimit:] # Training data, Test Data
    y_train, y_test = y[:splitlimit], y[splitlimit:] # Training data, Test Data

    # print("X_train.shape:", X_train.shape)
    # print("y_train.shape:", y_train.shape)
    # print("X_test.shape:", X_test.shape)
    # print("y_test.shape:", y_test.shape)
    # print("== X_train ===========================================")
    # print(X_train[-20:-10])
    # print("== _train ===========================================")
    # print(y_train[-20:-10])    

    return X_train, X_test, y_train, y_test

X_train, X_test, y_train, y_test = split_data(X, y, splitlimit)


# Load an Existing Model -or- Model Training
 - Using LSTM (Long Short Term Memory) Model
 - Using Keras (Tensorflow) Library
 - LSTM --> Dense Layer --> Activation Layer


In [9]:

import tensorflow as tf
import keras
import numpy as np
import os
from keras import optimizers
from keras.callbacks import History
from keras.models import Model
from keras.layers import Dense, Dropout, LSTM, Input, Activation, concatenate
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dropout
from keras.layers import Dense
from keras.layers import TimeDistributed

def create_fit_model(X_train, y_train, backcandels, model_path):
    # If the Keras model file exists, load it. Otherwise, create a new model.
    
    train_model = True
    if os.path.exists(model_path) and Selected_Model.value != CreateNewModel:
        # Load the model...
        model = keras.models.load_model(model_path)
        model_path = model_path.replace('~', '=')
        print("Model Loaded:", model_path)
        model_outputs = model.layers[3].output.shape[1]
        if model_outputs != 4:
            train_model = True
            print("** Model needs to be retrained.")
        else:
            train_model = False
            
    if train_model:
        # Create a new model...

        #tf.random.set_seed(20)
        np.random.seed(10)

        lstm_input = Input(shape=(backcandles, feature_cnt), name='lstm_input')
        inputs = LSTM(200, name='first_layer')(lstm_input)
        inputs = Dense(4, name='dense_layer')(inputs)
        output = Activation('linear', name='output')(inputs)
        model = Model(inputs=lstm_input, outputs=output)
        adam = optimizers.Adam()
        model.compile(optimizer=adam, loss='mse')
        # model.fit(x=X_train, y=y_train, batch_size=15, epochs=30, shuffle=True, validation_split = 0.1)
        model.fit(x=X_train, y=y_train, batch_size=250, epochs=150, shuffle=True, validation_split = 0.1)

        # Save the model...
        if chk_save_model.value:
            model_path = model_path.replace('=', '~')
            model.save(model_path)
    
    return model

model = create_fit_model(X_train, y_train, backcandles, model_path)


2024-08-28 06:40:30.529691: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-08-28 06:40:30.540867: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-08-28 06:40:30.554286: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-08-28 06:40:30.558257: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-08-28 06:40:30.568161: I tensorflow/core/platform/cpu_feature_guar

Model Loaded: models/EGIO_D_2020-08-31_2024-08-27.keras


I0000 00:00:1724852431.733496 2870265 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-08-28 06:40:31.758354: W tensorflow/core/common_runtime/gpu/gpu_device.cc:2343] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


# Model Testing

In [10]:
# Sow some y_pred values. If prediction is no good, we might see NaN values.
y_pred = model.predict(X_test)

# Rescaled the predicted values to dollars...
data_set_scaled_y = data_set_scaled[backcandles + splitlimit:, :].copy()
# Replace the last columns 4 in data_set_scaled_y with the predicted column values...
data_set_scaled_y[:, -4:] = y_pred

y_pred_rs = sc.inverse_transform(data_set_scaled_y)

pass

16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step


## - Adjust the prediction (Data Alignment with the actual previous prices)
  - Predictions have a tendency to wander way from the main price action channel.
      - The ideal for the prediction is for it to be as close to the actual price as possible.
      - What was observed is that the prediction is often off by a delta.
      - The delta is the difference between the actual price and the predicted price.
      - After applying an average of the delta to the prediction, the prediction is closer to the actual price.
      - Usually within 1% of the actual price, which is an incredible improvement.


  - Before we adjust the prediction, the data has to be rescaled to dollars because the each prediction target
    is scaled independently.
  
    - The resulting numpy array y_p_adj is the adjusted prediction, which we will use as the prediction for the next day.
 
 - The in the current implementation, Hign and Low adjusted predictions seem to have different characteristics.
      - I really want the High to be aligned with the actual high, yet preserving the trend of the prediction.
        But it seems that the adjusted high tends to wander upward from the prior high.
      - The Low, on the other hand, is quite close to the actual low, and has required different maths to adjust it.
      - I have tried getting the high and low maths be the same basic operation, but I haven't yet found the right formula.
        Thus, the difference in characteristics between the high and low adjusted predictions.
     - Nevertheless, for now what exists is workable.

In [11]:
# Calculate the delta between actual price and prediction
# Bring the prediction closer to the price based on the delta
import math

# Gather the target data for the test period.
target_close = np.array(data_set['Adj Close'].iloc[-len(y_pred_rs):])
target_high  = np.array(data_set['High'].iloc[-len(y_pred_rs):])
target_low   = np.array(data_set['Low'].iloc[-len(y_pred_rs):])

# Gather the predicted data for the test period.
pred_close = y_pred_rs[:, -3]
pred_high  = y_pred_rs[:, -2]
pred_low   = y_pred_rs[:, -1]

# Predictions are in y_red_rs
# Generate deltas between current prediction and prior prediction...
# -- Adjust Predicted Close
pred_delta_c = [pred_close[i - 1] - pred_close[i] for i in range(1, len(pred_close))]
min_len = min(len(target_close), len(pred_close))
target_close = target_close[-min_len:]
pred_adj_close = [target_close[i] + pred_delta_c[i] for i in range(0,len(pred_delta_c))]
# -- Adjust Predicted High
pred_delta_h = [pred_high[i - 1] - pred_high[i] for i in range(1, len(pred_high))]
min_len = min(len(target_high), len(pred_high))
target_high = target_high[-min_len:]
pred_adj_high = [target_high[i] + abs(pred_delta_h[i]) for i in range(0,len(pred_delta_h))]
#    -- Adjusted Close Prediction should not be higher than Adjusted High Prediction
pred_adj_high = [pred_adj_close[i] if pred_adj_close[i] > pred_adj_high[i] else pred_adj_high[i] for i in range(0,len(pred_delta_c))]
# -- Adjust Predicted low
pred_delta_c = [pred_low[i - 1] - pred_low[i] for i in range(1, len(pred_low))]
min_len = min(len(target_low), len(pred_low))
target_low = target_low[-min_len:]
pred_adj_low = [target_low[i] - abs(pred_delta_c[i]) for i in range(0,len(pred_delta_c))]
#    -- Adjusted Close Prediction should not be lower than Adjusted Low Prediction
pred_adj_low = [pred_adj_close[i] if pred_adj_close[i] < pred_adj_low[i] else pred_adj_low[i] for i in range(0,len(pred_delta_c))]

pass

# - Plot the Test Results

### - Rescaled (Target vs Prediction)

In [12]:
%matplotlib notebook

def plot_pred_results(target_close, pred_close, pred_high, pred_low):
    # Plot the scaled test and predicted data.
    y_test_plot = y_test[:-2, 1]
    plt.figure(figsize=(16,8))

    # Original Adj Close
    plt.plot(target_close,   color = 'black',  label = 'Test',       marker='.')

    # Original Prediction
    plt.plot(pred_close[1:], color = 'orange', label = 'Pred Close', marker='1')
    # Original High and Low prediction
    plt.plot(pred_high[1:],  color = 'blue',   label = 'Pred High',  marker='1')
    plt.plot(pred_low[1:],   color = 'red',    label = 'Pred Low',   marker='1')

    title = f"{ticker.value} ({dateStart.value} - {dateEnd.value})" 
    plt.title(title)
    plt.legend()
    plt.grid()
    plt.show()
    
plot_pred_results(target_close[1:-1], pred_close, pred_high, pred_low)

pass

<IPython.core.display.Javascript object>

### - Rescaled (Target vs Prediction vs Adjusted Prediction)

In [13]:
# Use an interactive plot to see the results
%matplotlib notebook

def plot_adj_pred_results(y_test, y_pred, pred_adj_close, pred_adj_high, pred_adj_low):
    # Plot the scaled test and predicted data.
    y_test_plot = y_test[:]
    plt.figure(figsize=(16,8))

    # Original Adj Close
    plt.plot(y_test_plot, color = 'black', label = 'Test',     marker='.')

    # # Original Prediction
    # plt.plot(y_pred[1:, 1],   color = 'yellow',  label = 'Pred Close', marker='1')
    # # Original High and Low prediction
    # plt.plot(y_pred[1:, 2],   color = 'blue',  label = 'Pred High', marker='1')
    # plt.plot(y_pred[1:, 3],   color = 'red',  label = 'Pred Low', marker='1')

    # Adjusted Predictions
    plt.plot(pred_adj_close,   color = 'yellow',  label = 'Pred Close', marker='1')
    # Original High and Low prediction
    plt.plot(pred_adj_high,   color = 'blue',  label = 'Pred High', marker='1')
    plt.plot(pred_adj_low,   color = 'red',  label = 'Pred Low', marker='1')

    title = f"{ticker.value} ({dateStart.value} - {dateEnd.value})"
    plt.title(title)
    plt.legend()
    plt.grid()
    plt.show()
    return plt

y_p_adj = None
plt = plot_adj_pred_results(target_close[1:-1], pred_close[1:], pred_adj_close[1:], pred_adj_high[1:], pred_adj_low[1:])
# pd.DataFrame(y_test_plot).tail(5)

<IPython.core.display.Javascript object>

### - Rescaled Data ($USD) (Prediction vs Target)

In [14]:
# Plot using candlesticks using matplotlib...

# Use an interactive plot to see the results
%matplotlib notebook

def plot_candlesticks(plt, prices, title, width=1, width2=.25, clr_up='green', clr_dn='red', clr_wick='black'):
    """
    Plot Candlesticks...
    :param plt:    maplotlib.pyplot handle
    :param prices: The series of OHLC prices to plot
    :param title:  A title for the plot
    :param width:
    :param width2:
    :param clr_up:
    :param clr_dn:
    :param clr_wick:
    :return:
    """
    #define up and down prices
    up = prices[prices.Close>=prices.Open]
    down = prices[prices.Close<prices.Open]

    #plot up prices
    plt.bar(up.index,up.Close-up.Open,width,bottom=up.Open,color=clr_up, alpha=.30)
    # plt.bar(up.index,up.High-up.Close,width2,bottom=up.Close,color=clr_up, alpha=.25)
    # plt.bar(up.index,up.Low-up.Open,width2,bottom=up.Open,color=clr_up, alpha=.25)
    plt.bar(up.index,up.High-up.Low,width2,bottom=up.Low,color=clr_wick, alpha=.30)

    #plot down prices
    plt.bar(down.index,down.Open-down.Close,width,bottom=down.Close,color=clr_dn, alpha=.25)
    # plt.bar(down.index,down.High-down.Open,width2,bottom=down.Open,color=clr_dn, alpha=.25)
    # plt.bar(down.index,down.Low-down.Close,width2,bottom=down.Close,color=clr_dn, alpha=.25)
    plt.bar(down.index,down.High-down.Low,width2,bottom=down.Low,color=clr_wick, alpha=.25)

    #rotate x-axis tick labels
    plt.xticks(rotation=45, ha='right')

import math
import mplfinance as mfp

def plot_pred_results(target_close, pred_close, y_pred_adj_rs, y_high_adj_rs, y_low_adj_rs, data_set, data_date, period):
    # Plot the original Adj Cost data that is in dollars...
    # Remove the last Adj Price, because it just aopy of the last adj
    plt_test_plot = np.array(target_close[:].reshape(-1,1))

    # Calculate the mean of the delta between the prediction and the actual price...
    # Used to create price range circles around the prediction...
    min_len = min(len(target_close), len(y_pred_adj_rs))
    plt_delta = np.array(pd.DataFrame([abs(target_close[i - 1] - y_pred_adj_rs[i - 1]) for i in range(1, min_len)]).rolling(window=6).mean())

    # Dates...
    data_date_str = [str(date) for date in data_date]
    plt_test_dates = np.array((data_date_str[backcandles + splitlimit - 1:]), ).reshape(-1,1)
    plt_pred_dates = np.array((data_date_str[backcandles + splitlimit - 1:]),   ).reshape(-1,1)

    # empty_arr = np.zeros(((len(plt_test_plot)+1, 18)))
    # plt_pred_adj = sc.inverse_transform(np.append(y_p_adj, empty_arr, axis=1))[0:, 1]

    # Plot the data...
    plt.figure(figsize=(16,8))
    plt.autoscale(tight=True)

    plt_test_plot = plt_test_plot.reshape(-1,1)

    # Plot the actual price data ($USD)...
    plt.plot(plt_test_plot, color = 'black', label = 'Actual Adj Close', marker='+')

    # Plot the adj predicted price data ($USD)...
    plt.plot(y_pred_adj_rs, color = 'orange', label = 'Adj Pred Close', marker='.')
    plt.plot(y_high_adj_rs, color = 'Blue', label = 'Adj Pred High', marker='.')
    plt.plot(y_low_adj_rs,  color = 'Red', label = 'Adj Pred Low', marker='.')

    # Plot the candlesticks...
    ds_tmp = data_set[backcandles + splitlimit + 1:-1]
    cs_data = ds_tmp.loc[backcandles + splitlimit:, ['Open','High','Low','Adj Close']].copy().reset_index()
    cs_data.rename(columns={'Adj Close':'Close'}, inplace=True)
    plot_candlesticks (plt, cs_data, 'Candlesticks')

    # Plot the text of the predicted price...
    plt.text(len(y_pred_adj_rs) -1, y_pred_adj_rs[-1], y_pred_adj_rs[-1].round(2), fontsize=8, color='black', ha='left', va='center')

    # plt.xlabel(range(0,len(plt_pred_dates)), plt_pred_dates.tolist(), rotation=45)
    #  .set_major_formatter(matplotlib.dates.DateFormatter('%Y')))

    # Plot a Price Range Circle Arounds the current Prediction...
    Plot_Pred_Circle = True
    if Plot_Pred_Circle:
        import matplotlib.patches as ptch
        lst_pred = len(y_pred_adj_rs) -1
        # ax.set_xlim = ([0, len(plt_pred_adj)])
        # ax.set_ylim = ([plt_pred_adj.min(), plt_pred_adj.max()])
        cpos = np.array([lst_pred, y_pred_adj_rs[-1]])
        circ_pred0 = ptch.Circle(cpos, radius=plt_delta[-1][0]/1,   color='grey',   fill=True, alpha=0.20)
        circ_pred1 = ptch.Circle(cpos, radius=plt_delta[-1][0]/1.5, color='blue',   fill=True, alpha=0.20)
        circ_pred2 = ptch.Circle(cpos, radius=plt_delta[-1][0]/2,   color='yellow', fill=True, alpha=0.20)
        circ_pred3 = ptch.Circle(cpos, radius=plt_delta[-1][0]/3,   color='red',    fill=True, alpha=0.20)
        ax = plt.gca()
        # ax.set_aspect("equal")
        ax.add_patch(circ_pred0)
        ax.add_patch(circ_pred1)
        ax.add_patch(circ_pred2)
        ax.add_patch(circ_pred3)

    title = f"{ticker.value} {period} ({dateStart.value} - {dateEnd.value})" 
    plt.title(title)
    plt.grid(visible=True, which='both')
    plt.legend()
    plt.show()
    return plt

period = PredsDaily
if '_w_' in model_path:
    period = PredsWeekly
data_set_scaled_plus = np.append(data_set_scaled, data_set_scaled[-1].reshape(1, -1), axis=0)
plt = plot_pred_results(target_close[1:-1], pred_close[1:], pred_adj_close[1:], pred_adj_high[1:], pred_adj_low[1:], data_set, data_date, period)


<IPython.core.display.Javascript object>

### - Plot the Prediction Results (Using mplfinance)

In [15]:
%matplotlib notebook
# Plot using mlpfinance...
import mplfinance as mpf
from matplotlib.widgets import MultiCursor

print('Ticker:', ticker.value)
def plot_pred_results(Ticker, data_orig, target_close, pred_close_rs, pred_close_adj_rs, pred_high_ad_rs, pred_low_ad_rs, pred_high_rs, pred_low_rs, data_date, period='Daily'):
    # Createe a dataframe of the data for plt_test_usd, with a datetime index...
    df_plt_test_usd = pd.DataFrame()
    df_plt_test_usd.insert(0, 'Date', data_date[backcandles + splitlimit:])
    df_plt_test_usd.reset_index()
    ohlcv = data_orig.iloc[backcandles + splitlimit:, [0,1,2,3,4,5]].copy()
    ohlcv.reset_index()
    df_plt_test_usd = pd.concat([df_plt_test_usd, ohlcv.set_axis(df_plt_test_usd.index)], axis=1)

    # Add a place holder for the prediction...
    next_date = df_plt_test_usd.iloc[-1, 0] + pd.DateOffset(days=1)
    next_open      = df_plt_test_usd.iloc[-1, 4]
    next_high      = df_plt_test_usd.iloc[-1, 4]
    next_low       = df_plt_test_usd.iloc[-1, 4]
    next_close     = df_plt_test_usd.iloc[-1, 4]
    next_adj_close = df_plt_test_usd.iloc[-1, 4]
    next_volume    = 0
    
    # Prepare the prediction data for the plot...
    c_pred_rs       = pd.DataFrame(pred_close_rs[:]).rename({0:'pred_rs'}, axis=1)
    c_pred_adj_rs   = pd.DataFrame(pred_close_adj_rs[:]).rename({0:'pred_adj_rs'}, axis=1)
    c_pred_h_adj_rs = pd.DataFrame(pred_high_ad_rs[:]).rename({0:'pred_h_adj_rs'}, axis=1)
    c_pred_l_adj_rs = pd.DataFrame(pred_low_ad_rs[:]).rename({0:'pred_l_adj_rs'}, axis=1)
    
    if pred_high_rs is not None and pred_low_rs is not None:
        c_pred_h_rs = pd.DataFrame(pred_high_rs).rename({0:'pred_h_rs'},  axis=1)
        c_pred_l_rs = pd.DataFrame(pred_low_rs).rename({0:'pred_l_rs'}, axis=1)
    
    # Append a new row to the dataframe so it extends out to the prediction...
    new_row = { 'Date': next_date, "Open": next_open, "High": next_high, "Low": next_low, "Close": next_close,
                                   "Adj Close": next_adj_close, "Volume": next_volume,
                                   "pred_rs": c_pred_rs.iloc[-1], "pred_adj_rs": c_pred_adj_rs.iloc[-1],
                                   "pred_h_adj_rs": c_pred_h_adj_rs.iloc[-1], "pred_l_adj_rs": c_pred_l_adj_rs.iloc[-1] }
    if pred_high_rs is not None and pred_low_rs is not None:
        new_row = { 'Date': next_date, "Open": next_open, "High": next_high, "Low": next_low, "Close": next_close,
                        "Adj Close": next_adj_close, "Volume": next_volume,
                        "pred_rs": c_pred_rs.iloc[-1], "pred_adj_rs": c_pred_adj_rs.iloc[-1],
                        "pred_h_adj_rs": c_pred_h_adj_rs.iloc[-1], "pred_l_adj_rs": c_pred_l_adj_rs.iloc[-1],
                        "pred_h_rs": c_pred_h_rs.iloc[-1], "pred_l_rs": c_pred_l_rs.iloc[-1] }
    else:
        new_row = { 'Date': next_date, "Open": next_open, "High": next_high, "Low": next_low, "Close": next_close,
                        "Adj Close": next_adj_close, "Volume": next_volume,
                        "pred_rs": c_pred_rs.iloc[-1], "pred_adj_rs": c_pred_adj_rs.iloc[-1],
                        "pred_h_adj_rs": c_pred_h_adj_rs.iloc[-1], "pred_l_adj_rs": c_pred_l_adj_rs.iloc[-1] }


    new_row = pd.DataFrame(new_row, index=[6])
    df_plt_test_usd = pd.concat([df_plt_test_usd[1:], new_row], axis=0, ignore_index=True)
        
    # # Add the unadjusted prediction data to the OHLCV plot data...
    # df_plt_test_usd = pd.concat([df_plt_test_usd, c_pred_rs[:len(df_plt_test_usd)].set_axis(df_plt_test_usd.index)],     axis=1)
    # if pred_high_rs is not None and pred_low_rs is not None:
    #     df_plt_test_usd = pd.concat([df_plt_test_usd, c_pred_h_rs[:len(df_plt_test_usd)].set_axis(df_plt_test_usd.index)], axis=1)
    #     df_plt_test_usd = pd.concat([df_plt_test_usd, c_pred_l_rs[:len(df_plt_test_usd)].set_axis(df_plt_test_usd.index)], axis=1)
    
    # Add adjusted prediction data to the OHLCV plot data...
    min_len = min(len(df_plt_test_usd), len(c_pred_h_adj_rs))
    df_plt_test_usd = df_plt_test_usd[-min_len:]
    df_plt_test_usd = pd.concat([df_plt_test_usd, c_pred_adj_rs[-min_len:].set_axis(df_plt_test_usd.index)], axis=1)
    df_plt_test_usd = pd.concat([df_plt_test_usd, c_pred_h_adj_rs[-min_len:].set_axis(df_plt_test_usd.index)], axis=1)
    df_plt_test_usd = pd.concat([df_plt_test_usd, c_pred_l_adj_rs[-min_len:].set_axis(df_plt_test_usd.index)], axis=1)
    
    df_plt_test_usd.ffill(inplace=True)
    df_plt_test_usd.set_index('Date', inplace=True, drop=True)
    df_plt_test_usd.ffill(inplace=True)
    df_plt_test_usd.rename(index={6:'pred_rs', 7:'pred_adj_rs'}, inplace=True)
    # Add a day row to df_plt_test_usd as a place holder for the prediction...
    print("df_plt_test_usd:", df_plt_test_usd.shape)
    
    kwargs = dict(type='candle', volume=True, figratio=(11,6), figscale=1.25, warn_too_much_data=10000, title=f'Ticker: {ticker.value} {period} ({df_plt_test_usd.index[-1]})')
    if pred_high_rs is not None and pred_low_rs is not None:
        preds = [ mpf.make_addplot(df_plt_test_usd['pred_adj_rs'],     type='line', panel=0, color='g', secondary_y=False),
                  mpf.make_addplot(df_plt_test_usd['pred_h_adj_rs'],   type='line', panel=0, color='b', secondary_y=False),
                  mpf.make_addplot(df_plt_test_usd['pred_l_adj_rs'],   type='line', panel=0, color='r', secondary_y=False),
                  mpf.make_addplot(df_plt_test_usd['pred_rs'],         type='line', linestyle='-.', panel=0, color='y', secondary_y=False),
                  mpf.make_addplot(df_plt_test_usd['pred_h_rs'],       type='line', linestyle='-.', panel=0, color='b', secondary_y=False),
                  mpf.make_addplot(df_plt_test_usd['pred_l_rs'],       type='line', linestyle='-.', panel=0, color='r', secondary_y=False),
                  ]
    else:
        preds = [ mpf.make_addplot(df_plt_test_usd['pred_adj_rs'],     type='line', panel=0, color='g', secondary_y=False),
                  mpf.make_addplot(df_plt_test_usd['pred_h_adj_rs'],   type='line', panel=0, color='b', secondary_y=False),
                  mpf.make_addplot(df_plt_test_usd['pred_l_adj_rs'],   type='line', panel=0, color='r', secondary_y=False),
                  ]

    fig, axlist = mpf.plot(df_plt_test_usd,**kwargs,style='binance', addplot=preds, returnfig=True)
    multi = MultiCursor(fig.canvas, axlist, color='r', lw=1, horizOn=True, vertOn=True)
    multi.connect()

    # # Given access to the Price chart at axis[0], we can use patches to draw shapes on the chart...
    # # The X axis is the number of periods on the chart (1 - N) and the Y axis is the price.
    # import matplotlib.patches as mptch
    # clipped_circle = mptch.Circle((len(df_plt_test_usd), 200), 5, ec="none", facecolor='C1')
    # axlist[0].add_patch(clipped_circle)

    return mpf

period = PredsDaily
if '_w_' in model_path:
    period = PredsWeekly
mpf = plot_pred_results(Ticker, data_orig, target_close, pred_close[1:], pred_adj_close[1:], pred_adj_high[1:], pred_adj_low[1:], pred_high[1:], pred_low[1:], data_date, period=period)


Ticker: EGIO
df_plt_test_usd: (481, 15)


<IPython.core.display.Javascript object>

# Analysis of the Prediction
 - Rescaled data (Prediction vs Target)

In [16]:
# We convert our data back to dollars...
# predval = y_pred[i] * scalefac

def analyze_pred(target_close, target_high, target_low, pred_close, pred_adj_close):
    # Convert back to dollar $values
    elements = len(target_close)
    print(elements)
    tot_deltas = 0
    tot_adj_deltas = 0
    tot_tradrng = 0
    for i in range(1, -elements, -1):
        actual = target_close[i - 1]
        
        predval = pred_close[i - 1]
        pred_delta = abs(predval - actual)
        tot_deltas += pred_delta

        predval = pred_adj_close[i - 1]
        pred_delta = abs(predval - actual)
        tot_deltas += pred_delta

        trd_rng = abs(target_high[i] - target_low[i])
        tot_tradrng += trd_rng
        print(f"i:{i}", "Close", actual.round(2), "Predicted: $", predval.round(2), "  Delta:$", pred_delta.round(6), "  Trade Rng: $", trd_rng.round(2))

    print("Mean Trading Range: $", round(tot_tradrng / elements, 2))
    print("Mean Delta (Actual vs Prediction): $", round((tot_deltas / elements), 2))
    print("Mean Delta (Actual vs Adj Prediction): $", round((tot_deltas / elements), 2))
    return tot_deltas, tot_tradrng, elements

tot_deltas, tot_tradrng, elements = analyze_pred(target_close, target_high, target_low, pred_close, pred_adj_close)


483
i:1 Close 113.2 Predicted: $ 106.68   Delta:$ 6.520264   Trade Rng: $ 2.8
i:0 Close 11.01 Predicted: $ 10.07   Delta:$ 0.942943   Trade Rng: $ 7.2
i:-1 Close 11.01 Predicted: $ 9.24   Delta:$ 1.766912   Trade Rng: $ 1.12
i:-2 Close 10.75 Predicted: $ 7.94   Delta:$ 2.809015   Trade Rng: $ 1.12
i:-3 Close 10.76 Predicted: $ 4.52   Delta:$ 6.238725   Trade Rng: $ 0.97
i:-4 Close 9.89 Predicted: $ 9.34   Delta:$ 0.54823   Trade Rng: $ 1.2
i:-5 Close 8.35 Predicted: $ 10.21   Delta:$ 1.858503   Trade Rng: $ 2.15
i:-6 Close 8.39 Predicted: $ 9.74   Delta:$ 1.353785   Trade Rng: $ 0.3
i:-7 Close 8.71 Predicted: $ 8.22   Delta:$ 0.492323   Trade Rng: $ 0.31
i:-8 Close 8.78 Predicted: $ 10.57   Delta:$ 1.787909   Trade Rng: $ 0.32
i:-9 Close 8.76 Predicted: $ 9.39   Delta:$ 0.633911   Trade Rng: $ 0.48
i:-10 Close 8.96 Predicted: $ 7.61   Delta:$ 1.354707   Trade Rng: $ 0.37
i:-11 Close 9.0 Predicted: $ 9.03   Delta:$ 0.032616   Trade Rng: $ 0.26
i:-12 Close 8.81 Predicted: $ 9.51   Delta:

IndexError: list index out of range

## - Rescaled data (0..1) (Adjusted Prediction vs Target)

In [ ]:

def analyze_pred(target_close, target_high, target_low, pred_close):
    # Convert back to dollar $values
    elements = len(target_close)
    print(elements)
    tot_deltas = 0
    tot_tradrng = 0
    for i in range(1, -elements, -1):
        actual = target_close[i - 1]
        scalefac = target_close[i - 1] / target_close[i - 1]
        # print("Scaling Factor", scalefac)
        predval = pred_close[i] * scalefac
        predval = pred_close[i - 1] if np.isinf(predval) else predval
        pred_delta = abs(predval - actual)
        tot_deltas += pred_delta
        trd_rng = abs(target_high[i] - target_low[i])
        tot_tradrng += trd_rng
        print(f"i:{i}", "Close", actual.round(2), "Predicted: $", predval.round(2), "  Delta:$", pred_delta.round(6), "  Trade Rng: $", trd_rng.round(2))

    print("Mean Trading Range: $", round(tot_tradrng / elements, 2))
    print("Mean Delta (Actual vs Prediction): $", round((tot_deltas / elements), 2))
    return tot_deltas, tot_tradrng, elements

tot_deltas, tot_tradrng, elements = analyze_pred(target_close, target_high, target_low, pred_adj_close)

print("Mean Trading Range: $", round(tot_tradrng / elements, 2))
print("Mean Delta (Actual vs Prediction): $", round((tot_deltas[0] / elements), 2))

### - Deltas between predicted price and actual price

In [ ]:
# Calculate an array of deltas between predicted price and actual price
print("Std Dev -     Pred: ", pred_delta.std())
y_pa_delta = abs(y_test - y_p_adj)
print("Std Dev - Pred Adj: ", y_pa_delta.std())
print("Better by", (y_p_delta.std() / y_pa_delta.std()), "Standard Deviations")

### - Deltas between predicted price and actual price

In [ ]:
# Calculate an array of deltas between predicted price and actual price
print("Std Dev -     Pred: ", pred_delta.std())
y_pa_delta = abs(y_test - y_p_adj)
print("Std Dev - Pred Adj: ", y_pa_delta.std())
print("Better by", (y_p_delta.std() / y_pa_delta.std()), "Standard Deviations")

## - Trend Prediction Stats using Unscaled (v < 1) Adjusted 
** The stats are not entirely accurate a the adjusted prediction will change when the new close actually occurs 
   and the adjusting process includes the new close in the calculation


In [ ]:
# How often is the prediction within 1% of the actual price?
pct = 0.05
pct_delta = abs(y_test * pct)
delta = abs(y_test - y_p_adj)
within_range = delta[delta < pct_delta]
# Get average price range for the instrument
avg_price_rng = data_set['High'] - data_set['Low']
# Get pct of average price range
pcnt_avg_price_rng = avg_price_rng.mean() * pct

print("Within",pct * 100,"% of Acutal Adj Close","[ Trading Range:$", round(tot_tradrng / elements, 2), 
      " Within Avg Price:", avg_price_rng.mean().round(2), " Average Difference to Actual: $", pcnt_avg_price_rng.round(2),"]", 
      " Pct within Range:", round((len(within_range) / len(y_test)),2) * 100,"%")

# How often is the predicted trend the same as the actual trend?
test_trend = y_test[1:] - y_test[:-1]
pred_trend = y_p_adj[1:] - y_p_adj[:-1]
delta = [1 if (test_trend < 0) and (pred_trend < 0) or (test_trend > 0) and (pred_trend > 0) else 0 for test_trend, pred_trend in zip(test_trend, pred_trend)]
# Print the number of elements in delta and the percent of delta that are 1.
print("Same Trend:", len(delta), "Wrong:", len(delta) - len(within_range)+1, "Right", np.count_nonzero(within_range+1), "%Right", round((len(within_range)-1) / len(delta) * 100,2))


# Model Details...

In [ ]:
# Review feature importance
# importance = model.coef_

import shap
import pydot

print(X_train.shape)

print(model.summary())
print("model.outputs", model.layers[3].output.shape[1])
from keras.utils import plot_model
plot_model(model, to_file='model_plot.png', show_shapes=True, show_layer_names=True)
from IPython.display import Image

pass

## - Model Plot
<img src="model_plot.png" alt="Drawing" style="width: 200px;"/>